# Age prediction using the top 60 EC epochs found in del2_2 (calculating alpha power)
### Predicting age groups with increments of 5

In [ ]:
import mne
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# ------------ LOAD DATA ------------
with open("top_epochs_per_subject.pkl", "rb") as f:
    top_epochs_per_subject = pickle.load(f)

# Ensure keys are strings and trimmed
top_epochs_per_subject = {str(k).strip(): v for k, v in top_epochs_per_subject.items()}

metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str).str.strip()

# Filter out subjects 90 years or older
metadata = metadata[metadata["age"] < 90]

# Keep only subjects present in both metadata and epochs dictionary
all_subjects = [s for s in top_epochs_per_subject.keys() if s in metadata["subject_id"].values]

random.seed(42)
test_subjects = random.sample(all_subjects, 500)
train_subjects = [s for s in all_subjects if s not in test_subjects]

# ------------ DEFINE AGE GROUP LABELS ------------
def assign_age_group(age):
    return age // 10  # Groups by decades: 0–9 → 0, 10–19 →1, etc.

metadata["age_group"] = metadata["age"].apply(assign_age_group)
metadata = metadata[metadata["subject_id"].isin(all_subjects)]

# ------------ FEATURE EXTRACTION ------------
def extract_combined_features(subject_id, epoch_indices, set_folder):
    path = f"{set_folder}/{subject_id}_epoched.set"
    epochs = mne.io.read_epochs_eeglab(path, verbose='ERROR')
    data = epochs.get_data()[epoch_indices]
    sfreq = epochs.info["sfreq"]

    bands = {
        "delta": (1, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "beta": (13, 30),
    }
    
    band_powers = []
    relative_powers = []

    # Total PSD for 1-45 Hz (for relative power & slope)
    total_psds, total_freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45, n_fft=128, verbose=False
    )
    total_power = total_psds.sum(axis=-1)  # shape: (epochs, channels)

    # --- Spectral slope calculation ---
    mean_psd = total_psds.mean(axis=(0,1))  # mean over epochs and channels
    freqs_log = np.log10(total_freqs)
    psd_log = np.log10(mean_psd)

    model = LinearRegression()
    model.fit(freqs_log.reshape(-1,1), psd_log)
    spectral_slope = model.coef_[0]

    # --- Alpha peak frequency ---
    psds_alpha, freqs_alpha = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=8, fmax=13, n_fft=128, verbose=False
    )
    mean_alpha_psd = psds_alpha.mean(axis=(0,1))
    alpha_peak_idx = mean_alpha_psd.argmax()
    alpha_peak_freq = freqs_alpha[alpha_peak_idx]

    # Compute band powers and relative powers
    for band_name, (fmin, fmax) in bands.items():
        psds_band, _ = mne.time_frequency.psd_array_welch(
            data, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=128, verbose=False
        )
        abs_power = psds_band.mean(axis=(0,2))  # mean over epochs and freq bins per channel
        band_powers.extend(abs_power)
        
        rel_power = (psds_band.sum(axis=-1) / total_power).mean()
        relative_powers.append(rel_power)
    
    features = np.concatenate([band_powers, relative_powers, [spectral_slope, alpha_peak_freq]])
    return features

# ------------ EXTRACT FEATURES FROM TRAIN SET ------------
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # <-- Update your path here!

X, y, subject_ids = [], [], []

for subj_id in train_subjects:
    try:
        epoch_inds = top_epochs_per_subject[subj_id]
        features = extract_combined_features(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X.append(features)
        y.append(age_group)
        subject_ids.append(subj_id)
    except Exception as e:
        print(f" Error processing {subj_id}: {e}")

X = np.array(X)
y = np.array(y)

# ------------ 5-FOLD CROSS VALIDATION ------------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), start=1):
    print(f"\n Fold {fold} running...")
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model = RandomForestClassifier(n_estimators=200, random_state=42)
    model.fit(X_train_scaled, y_train)

    preds = model.predict(X_val_scaled)
    acc = accuracy_score(y_val, preds)
    accuracies.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.3f}")

print(f"\n Mean CV Accuracy: {np.mean(accuracies):.3f}")

# ------------ FINAL MODEL TRAINING ------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
final_model = RandomForestClassifier(n_estimators=200, random_state=42)
final_model.fit(X_scaled, y)

# ------------ TEST SET EVALUATION ------------
X_test, y_test = [], []

for subj_id in test_subjects:
    try:
        epoch_inds = top_epochs_per_subject[subj_id]
        features = extract_combined_features(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X_test.append(features)
        y_test.append(age_group)
    except Exception as e:
        print(f" Error processing test subject {subj_id}: {e}")

X_test = np.array(X_test)
y_test = np.array(y_test)
X_test_scaled = scaler.transform(X_test)

test_preds = final_model.predict(X_test_scaled)
test_acc = accuracy_score(y_test, test_preds)
print(f"\n Final Test Accuracy: {test_acc:.3f}")
print(classification_report(y_test, test_preds))


In [ ]:
print(classification_report(y_test, test_preds))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, test_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
fig, ax = plt.subplots(figsize=(8,8))
disp.plot(ax=ax, cmap = 'Blues', colorbar = True)
plt.title("Confusion Matrix for Test Set Predictions")


# The same kind of classification, but instead using 60 radnmly chosen epochs from label_predictions 

### Extracting 60 random epochs 

In [ ]:
labels_df = pd.read_csv("label_predictions.csv")  

# Sample 60 epochs per subject (mixed EC and EO)
random_epochs_per_subject = {}
for subj_id, group in labels_df.groupby("Test subject ID"):
    if len(group) >= 60:
        sampled = group.sample(n=60, random_state=13)
        random_epochs_per_subject[subj_id] = sampled["Epoch number"].values

In [ ]:
len(random_epochs_per_subject)

In [ ]:
import mne
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str).str.strip()

# Filter out subjects 90 or older
metadata = metadata[metadata["age"] < 90]

random_epochs_per_subject = {str(k).strip(): v for k, v in random_epochs_per_subject.items()}

# Keep only subjects present in both metadata and random_epochs_per_subject
all_subjects_random = [s for s in random_epochs_per_subject.keys() if s in metadata["subject_id"].values]
print(len(all_subjects_random))

random.seed(42)
test_subjects_random = test_subjects
train_subjects_random = [s for s in all_subjects_random if s not in test_subjects_random]

# Define age groups
def assign_age_group(age):
    return age // 10  # decade groups

metadata["age_group"] = metadata["age"].apply(assign_age_group)
metadata = metadata[metadata["subject_id"].isin(all_subjects_random)]

# ------------ FEATURE EXTRACTION ------------
def extract_combined_features_random(subject_id, epoch_indices, set_folder):
    path = f"{set_folder}/{subject_id}_epoched.set"
    epochs = mne.io.read_epochs_eeglab(path, verbose='ERROR')
    data = epochs.get_data()[epoch_indices]
    sfreq = epochs.info["sfreq"]

    bands = {
        "delta": (1, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "beta": (13, 30),
    }
    
    band_powers = []
    relative_powers = []

    # Total PSD for 1-45 Hz (for relative power & slope)
    total_psds, total_freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45, n_fft=128, verbose=False
    )
    total_power = total_psds.sum(axis=-1)  # shape: (epochs, channels)

    # Spectral slope calculation
    mean_psd = total_psds.mean(axis=(0,1))  # mean over epochs and channels
    freqs_log = np.log10(total_freqs)
    psd_log = np.log10(mean_psd)

    model = LinearRegression()
    model.fit(freqs_log.reshape(-1,1), psd_log)
    spectral_slope = model.coef_[0]

    # Alpha peak frequency
    psds_alpha, freqs_alpha = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=8, fmax=13, n_fft=128, verbose=False
    )
    mean_alpha_psd = psds_alpha.mean(axis=(0,1))
    alpha_peak_idx = mean_alpha_psd.argmax()
    alpha_peak_freq = freqs_alpha[alpha_peak_idx]

    # Band powers & relative powers
    for band_name, (fmin, fmax) in bands.items():
        psds_band, _ = mne.time_frequency.psd_array_welch(
            data, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=128, verbose=False
        )
        abs_power = psds_band.mean(axis=(0,2))
        band_powers.extend(abs_power)
        
        rel_power = (psds_band.sum(axis=-1) / total_power).mean()
        relative_powers.append(rel_power)
    
    features = np.concatenate([band_powers, relative_powers, [spectral_slope, alpha_peak_freq]])
    return features

# ------------ EXTRACT FEATURES FROM TRAIN SET ------------
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # <-- Update your path here!

X_random, y_random, subject_ids_random = [], [], []

for subj_id in train_subjects_random:
    try:
        epoch_inds = random_epochs_per_subject[subj_id]
        features = extract_combined_features_random(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X_random.append(features)
        y_random.append(age_group)
        subject_ids_random.append(subj_id)
    except Exception as e:
        print(f" Error processing {subj_id}: {e}")

X_random = np.array(X_random)
y_random = np.array(y_random)

# ------------ 5-FOLD CROSS VALIDATION ------------
skf_random = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies_random = []

for fold, (train_idx, val_idx) in enumerate(skf_random.split(X_random, y_random), start=1):
    print(f"\n Fold {fold} running...")
    X_train, X_val = X_random[train_idx], X_random[val_idx]
    y_train, y_val = y_random[train_idx], y_random[val_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model_random = RandomForestClassifier(n_estimators=200, random_state=42)
    model_random.fit(X_train_scaled, y_train)

    preds_random = model_random.predict(X_val_scaled)
    acc = accuracy_score(y_val, preds_random)
    accuracies_random.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.3f}")

print(f"\n Mean CV Accuracy: {np.mean(accuracies_random):.3f}")

# ------------ FINAL MODEL TRAINING ------------
scaler = StandardScaler()
X_scaled_random = scaler.fit_transform(X_random)
final_model_random = RandomForestClassifier(n_estimators=200, random_state=42)
final_model_random.fit(X_scaled_random, y_random)

# ------------ TEST SET EVALUATION ------------
X_test_random, y_test_random = [], []

for subj_id in test_subjects_random:
    try:
        epoch_inds = random_epochs_per_subject[subj_id]
        features = extract_combined_features_random(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X_test_random.append(features)
        y_test_random.append(age_group)
    except Exception as e:
        print(f"Error processing test subject {subj_id}: {e}")

X_test_random = np.array(X_test_random)
y_test_random = np.array(y_test_random)
X_test_scaled_random = scaler.transform(X_test_random)

test_preds_random = final_model_random.predict(X_test_scaled_random)
test_acc_random = accuracy_score(y_test_random, test_preds_random)
print(f"\n Final Test Accuracy (Random Epochs): {test_acc_random:.3f}")
print(classification_report(y_test_random, test_preds_random))


In [ ]:
print(classification_report(y_test_random, test_preds_random))

In [ ]:
cm_random = confusion_matrix(y_test_random, test_preds_random)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_random)
fig, ax = plt.subplots(figsize=(8,8))
disp.plot(ax=ax, cmap = 'Blues', colorbar = True)
plt.title("Confusion Matrix for Test Set Predictions - random epochs")

In [ ]:
import pandas as pd
import pickle 

# Load the label predictions CSV
labels_df = pd.read_csv("label_predictions.csv")

# Dictionary to hold top 60 EO epochs per subject
top_60_EO_epochs_per_subject = {}

# Group by subject ID (assuming the column is 'subject_id')
for subject_id, group in labels_df.groupby("Test subject ID"):
    # Filter epochs labeled as eyes-open (label == 0)
    eyes_open_epochs = group[group["Label"] == 0]

    # Sort by probability descending to get highest confidence epochs first
    eyes_open_sorted = eyes_open_epochs.sort_values(by="Probability", ascending=False)

    # Take top 60 epoch numbers (or fewer if less than 60 available)
    top_epochs = eyes_open_sorted.head(60)["Epoch number"].values

    # Store in dictionary
    top_60_EO_epochs_per_subject[subject_id] = top_epochs

with open("top_60_EO_epochs_per_subject.pkl", "wb") as f:
    pickle.dump(top_60_EO_epochs_per_subject, f)


In [ ]:
import mne
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# ------------ LOAD METADATA ------------
metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str).str.strip()

# Filter out subjects 90 or older
metadata = metadata[metadata["age"] < 90]

# Use your previously created dictionary of top 60 EO epochs per subject
# Make sure keys are strings and trimmed
top_60_EO_epochs_per_subject = {str(k).strip(): v for k, v in top_60_EO_epochs_per_subject.items()}

# Keep only subjects present in both metadata and EO dict
all_subjects_EO = [s for s in top_60_EO_epochs_per_subject.keys() if s in metadata["subject_id"].values]
print(f"Number of EO subjects: {len(all_subjects_EO)}")

# Use the SAME test_subjects list from random epochs code to ensure consistent hold-out set
# (Make sure this list 'test_subjects' is already defined and contains subject IDs as strings)
test_subjects_EO = test_subjects
train_subjects_EO = [s for s in all_subjects_EO if s not in test_subjects_EO]

# Define age groups (decades)
def assign_age_group(age):
    return age // 10

metadata["age_group"] = metadata["age"].apply(assign_age_group)
metadata = metadata[metadata["subject_id"].isin(all_subjects_EO)]

# ------------ FEATURE EXTRACTION ------------
def extract_combined_features_EO(subject_id, epoch_indices, set_folder):
    path = f"{set_folder}/{subject_id}_epoched.set"
    epochs = mne.io.read_epochs_eeglab(path, verbose='ERROR')
    data = epochs.get_data()[epoch_indices]
    sfreq = epochs.info["sfreq"]

    bands = {
        "delta": (1, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "beta": (13, 30),
    }
    
    band_powers = []
    relative_powers = []

    # Total PSD for 1-45 Hz (for relative power & spectral slope)
    total_psds, total_freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45, n_fft=128, verbose=False
    )
    total_power = total_psds.sum(axis=-1)  # shape: (epochs, channels)

    # Spectral slope calculation
    mean_psd = total_psds.mean(axis=(0,1))  # mean over epochs and channels
    freqs_log = np.log10(total_freqs)
    psd_log = np.log10(mean_psd)

    model = LinearRegression()
    model.fit(freqs_log.reshape(-1,1), psd_log)
    spectral_slope = model.coef_[0]

    # Alpha peak frequency
    psds_alpha, freqs_alpha = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=8, fmax=13, n_fft=128, verbose=False
    )
    mean_alpha_psd = psds_alpha.mean(axis=(0,1))
    alpha_peak_idx = mean_alpha_psd.argmax()
    alpha_peak_freq = freqs_alpha[alpha_peak_idx]

    # Band powers & relative powers
    for band_name, (fmin, fmax) in bands.items():
        psds_band, _ = mne.time_frequency.psd_array_welch(
            data, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=128, verbose=False
        )
        abs_power = psds_band.mean(axis=(0,2))
        band_powers.extend(abs_power)
        
        rel_power = (psds_band.sum(axis=-1) / total_power).mean()
        relative_powers.append(rel_power)
    
    features = np.concatenate([band_powers, relative_powers, [spectral_slope, alpha_peak_freq]])
    return features

# ------------ EXTRACT FEATURES FROM TRAIN SET ------------
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # <-- Update this path

X_EO, y_EO, subject_ids_EO = [], [], []

for subj_id in train_subjects_EO:
    try:
        epoch_inds = top_60_EO_epochs_per_subject[subj_id]
        features = extract_combined_features_EO(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X_EO.append(features)
        y_EO.append(age_group)
        subject_ids_EO.append(subj_id)
    except Exception as e:
        print(f" Error processing {subj_id}: {e}")

X_EO = np.array(X_EO)
y_EO = np.array(y_EO)

# ------------ 5-FOLD CROSS VALIDATION ------------
skf_EO = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies_EO = []

for fold, (train_idx, val_idx) in enumerate(skf_EO.split(X_EO, y_EO), start=1):
    print(f"\n Fold {fold} running...")
    X_train, X_val = X_EO[train_idx], X_EO[val_idx]
    y_train, y_val = y_EO[train_idx], y_EO[val_idx]

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model_EO = RandomForestClassifier(n_estimators=200, random_state=42)
    model_EO.fit(X_train_scaled, y_train)

    preds_EO = model_EO.predict(X_val_scaled)
    acc = accuracy_score(y_val, preds_EO)
    accuracies_EO.append(acc)
    print(f"Fold {fold} Accuracy: {acc:.3f}")

print(f"\n Mean CV Accuracy (EO epochs): {np.mean(accuracies_EO):.3f}")

# ------------ FINAL MODEL TRAINING ------------
scaler = StandardScaler()
X_scaled_EO = scaler.fit_transform(X_EO)
final_model_EO = RandomForestClassifier(n_estimators=200, random_state=42)
final_model_EO.fit(X_scaled_EO, y_EO)

# ------------ TEST SET EVALUATION ------------
X_test_EO, y_test_EO = [], []

for subj_id in test_subjects_EO:
    try:
        epoch_inds = top_60_EO_epochs_per_subject[subj_id]
        features = extract_combined_features_EO(subj_id, epoch_inds, set_folder)
        age_group = metadata.loc[metadata["subject_id"] == subj_id, "age_group"].values[0]
        X_test_EO.append(features)
        y_test_EO.append(age_group)
    except Exception as e:
        print(f" Error processing test subject {subj_id}: {e}")

X_test_EO = np.array(X_test_EO)
y_test_EO = np.array(y_test_EO)
X_test_scaled_EO = scaler.transform(X_test_EO)

test_preds_EO = final_model_EO.predict(X_test_scaled_EO)
test_acc_EO = accuracy_score(y_test_EO, test_preds_EO)
print(f"\n Final Test Accuracy (EO epochs): {test_acc_EO:.3f}")
print(classification_report(y_test_EO, test_preds_EO))


In [ ]:
print(classification_report(y_test_EO, test_preds_EO))

In [ ]:
cm_EO = confusion_matrix(y_test_EO, test_preds_EO)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_EO)
fig, ax = plt.subplots(figsize=(8,8))
disp.plot(ax=ax, cmap = 'Blues', colorbar = True)
plt.title("Confusion Matrix for Test Set Predictions - Eyes Open Epochs")

# Plotting the mean absolute alpha power for those same 60 random epochs as chosen before. 

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch
from scipy.interpolate import make_interp_spline

# ------------ Load metadata ------------
metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str)

# ------------ Compute mean alpha power for random epochs ------------
alpha_by_subject_random = {}
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # Update this path

for subject_id in random_epochs_per_subject:
    try:
        set_path = f"{set_folder}/{subject_id}_epoched.set"
        epochs = mne.io.read_epochs_eeglab(set_path, verbose='ERROR')
        data = epochs.get_data()[random_epochs_per_subject[subject_id]]

        psds, freqs = psd_array_welch(
            data,
            sfreq=epochs.info["sfreq"],
            fmin=8, fmax=13,
            n_fft=200,
            verbose=False
        )

        mean_power_per_channel = psds.mean(axis=-1)
        mean_power_over_epochs = mean_power_per_channel.mean(axis=0)
        alpha_value = mean_power_over_epochs.mean() * 1e12  # µV²/Hz

        alpha_by_subject_random[subject_id] = alpha_value

    except Exception as e:
        print(f" Could not process {subject_id}: {e}")

# ------------ Merge alpha values with metadata ------------
alpha_df = pd.DataFrame({
    "subject_id": list(alpha_by_subject_random.keys()),
    "alpha_power": list(alpha_by_subject_random.values())
})

metadata_subset = metadata[metadata["subject_id"].isin(alpha_df["subject_id"])]

alpha_df["subject_id"] = alpha_df["subject_id"].astype(str)
metadata_subset["subject_id"] = metadata_subset["subject_id"].astype(str)

merged = pd.merge(alpha_df, metadata_subset, on="subject_id")

# Filter out extreme alpha power values
merged = merged[merged["alpha_power"] <= 20]

# ------------ Group by age ------------
grouped = merged.groupby("age").agg(
    mean_alpha=("alpha_power", "mean"),
    std=("alpha_power", "std"),
    N=("alpha_power", "count")
).reset_index()

grouped["sem"] = grouped["std"] / np.sqrt(grouped["N"])
grouped["ci_upper"] = grouped["mean_alpha"] + 1.96 * grouped["sem"]
grouped["ci_lower"] = grouped["mean_alpha"] - 1.96 * grouped["sem"]

# Remove NaNs/Infs for smoothing
valid_idx = (~grouped["ci_upper"].isna()) & (~grouped["ci_lower"].isna()) & \
            (~grouped["ci_upper"].isin([np.inf, -np.inf])) & (~grouped["ci_lower"].isin([np.inf, -np.inf]))

ages = grouped.loc[valid_idx, "age"].values
mean_alpha = grouped.loc[valid_idx, "mean_alpha"].values
ci_upper = grouped.loc[valid_idx, "ci_upper"].values
ci_lower = grouped.loc[valid_idx, "ci_lower"].values

# Create smooth x values
ages_smooth = np.linspace(ages.min(), ages.max(), 500)

# Fit splines (degree k=5)
mean_spline = make_interp_spline(ages, mean_alpha, k=5)
ci_upper_spline = make_interp_spline(ages, ci_upper, k=5)
ci_lower_spline = make_interp_spline(ages, ci_lower, k=5)

# Evaluate splines
mean_smooth = mean_spline(ages_smooth)
ci_upper_smooth = ci_upper_spline(ages_smooth)
ci_lower_smooth = ci_lower_spline(ages_smooth)

# ------------ Plot ------------
plt.figure(figsize=(14, 5))
plt.plot(ages_smooth, mean_smooth, label="Smoothed Mean", color="blue")
plt.fill_between(ages_smooth, ci_lower_smooth, ci_upper_smooth, color="skyblue", alpha=0.4, label="95% CI")
plt.scatter(ages, mean_alpha, color="blue", s=10, label="Mean Alpha Power")

plt.xlabel("Age")
plt.ylabel("Mean Alpha Power (µV²/Hz)")
plt.title("Mean Absolute Alpha Power vs. Age (Random Epochs) with 95% Confidence Interval")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Plotting the mean relative alpha power for those same 60 random epochs as chosen before. 

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch
from scipy.interpolate import make_interp_spline

# ------------ Load metadata ------------
metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str)

# ------------ Compute mean relative alpha power for random epochs ------------
relative_alpha_by_subject = {}
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # Update this path

for subject_id in random_epochs_per_subject:
    try:
        set_path = f"{set_folder}/{subject_id}_epoched.set"
        epochs = mne.io.read_epochs_eeglab(set_path, verbose='ERROR')
        selected_data = epochs.get_data()[random_epochs_per_subject[subject_id]]

        # Alpha power (8–13 Hz)
        psds_alpha, _ = psd_array_welch(
            selected_data,
            sfreq=epochs.info["sfreq"],
            fmin=8, fmax=13,
            n_fft=200,
            verbose=False
        )
        alpha_power = psds_alpha.sum(axis=-1).mean()  # sum over freq bins, mean over epochs & channels

        # Total power (1–40 Hz)
        psds_total, _ = psd_array_welch(
            selected_data,
            sfreq=epochs.info["sfreq"],
            fmin=1, fmax=40,
            n_fft=200,
            verbose=False
        )
        total_power = psds_total.sum(axis=-1).mean()

        # Relative alpha power
        relative_alpha = alpha_power / total_power
        relative_alpha_by_subject[subject_id] = relative_alpha

    except Exception as e:
        print(f" Could not process {subject_id}: {e}")

# ------------ Merge relative alpha values with metadata ------------
alpha_df = pd.DataFrame({
    "subject_id": list(relative_alpha_by_subject.keys()),
    "relative_alpha_power": list(relative_alpha_by_subject.values())
})

metadata_subset = metadata[metadata["subject_id"].isin(alpha_df["subject_id"])]

alpha_df["subject_id"] = alpha_df["subject_id"].astype(str)
metadata_subset["subject_id"] = metadata_subset["subject_id"].astype(str)

merged = pd.merge(alpha_df, metadata_subset, on="subject_id")

# Filter out extreme relative alpha values (optional, adjust threshold as needed)
merged = merged[merged["relative_alpha_power"] <= 1.0]

# ------------ Group by age ------------
grouped = merged.groupby("age").agg(
    mean_rel_alpha=("relative_alpha_power", "mean"),
    std=("relative_alpha_power", "std"),
    N=("relative_alpha_power", "count")
).reset_index()

grouped["sem"] = grouped["std"] / np.sqrt(grouped["N"])
grouped["ci_upper"] = grouped["mean_rel_alpha"] + 1.96 * grouped["sem"]
grouped["ci_lower"] = grouped["mean_rel_alpha"] - 1.96 * grouped["sem"]

# Remove NaNs/Infs for smoothing
valid_idx = (~grouped["ci_upper"].isna()) & (~grouped["ci_lower"].isna()) & \
            (~grouped["ci_upper"].isin([np.inf, -np.inf])) & (~grouped["ci_lower"].isin([np.inf, -np.inf]))

ages = grouped.loc[valid_idx, "age"].values
mean_rel_alpha = grouped.loc[valid_idx, "mean_rel_alpha"].values
ci_upper = grouped.loc[valid_idx, "ci_upper"].values
ci_lower = grouped.loc[valid_idx, "ci_lower"].values

# Create smooth x values
ages_smooth = np.linspace(ages.min(), ages.max(), 500)

# Fit splines (degree k=5)
mean_spline = make_interp_spline(ages, mean_rel_alpha, k=5)
ci_upper_spline = make_interp_spline(ages, ci_upper, k=5)
ci_lower_spline = make_interp_spline(ages, ci_lower, k=5)

# Evaluate splines
mean_smooth = mean_spline(ages_smooth)
ci_upper_smooth = ci_upper_spline(ages_smooth)
ci_lower_smooth = ci_lower_spline(ages_smooth)

# ------------ Plot ------------
plt.figure(figsize=(14, 5))
plt.plot(ages_smooth, mean_smooth, label="Smoothed Mean Relative Alpha", color="blue")
plt.fill_between(ages_smooth, ci_lower_smooth, ci_upper_smooth, color="skyblue", alpha=0.4, label="95% CI")
plt.scatter(ages, mean_rel_alpha, color="blue", s=10, label="Mean Relative Alpha")

plt.xlabel("Age")
plt.ylabel("Mean Relative Alpha Power")
plt.title("Mean Relative Alpha Power vs. Age (Random Epochs) with 95% Confidence Interval")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Plotting absolute mean alpha for 60 eyes open epochs per subject

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch
from scipy.interpolate import make_interp_spline

# ------------ Load metadata ------------
metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str)

# ------------ Compute mean alpha power for random epochs ------------
alpha_by_subject_random = {}
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # Update this path

for subject_id in top_60_EO_epochs_per_subject:
    try:
        set_path = f"{set_folder}/{subject_id}_epoched.set"
        epochs = mne.io.read_epochs_eeglab(set_path, verbose='ERROR')
        data = epochs.get_data()[top_60_EO_epochs_per_subject[subject_id]]

        psds, freqs = psd_array_welch(
            data,
            sfreq=epochs.info["sfreq"],
            fmin=8, fmax=13,
            n_fft=200,
            verbose=False
        )

        mean_power_per_channel = psds.mean(axis=-1)
        mean_power_over_epochs = mean_power_per_channel.mean(axis=0)
        alpha_value = mean_power_over_epochs.mean() * 1e12  # µV²/Hz

        alpha_by_subject_random[subject_id] = alpha_value

    except Exception as e:
        print(f" Could not process {subject_id}: {e}")



In [ ]:
# ------------ Merge alpha values with metadata ------------
alpha_df = pd.DataFrame({
    "subject_id": list(alpha_by_subject_random.keys()),
    "alpha_power": list(alpha_by_subject_random.values())
})

metadata_subset = metadata[metadata["subject_id"].isin(alpha_df["subject_id"])]

alpha_df["subject_id"] = alpha_df["subject_id"].astype(str)
metadata_subset["subject_id"] = metadata_subset["subject_id"].astype(str)

merged = pd.merge(alpha_df, metadata_subset, on="subject_id")

# Filter out extreme alpha power values
merged = merged[merged["alpha_power"] <= 20]

# ------------ Group by age ------------
grouped = merged.groupby("age").agg(
    mean_alpha=("alpha_power", "mean"),
    std=("alpha_power", "std"),
    N=("alpha_power", "count")
).reset_index()

grouped["sem"] = grouped["std"] / np.sqrt(grouped["N"])
grouped["ci_upper"] = grouped["mean_alpha"] + 1.96 * grouped["sem"]
grouped["ci_lower"] = grouped["mean_alpha"] - 1.96 * grouped["sem"]

# Remove NaNs/Infs for smoothing
valid_idx = (~grouped["ci_upper"].isna()) & (~grouped["ci_lower"].isna()) & \
            (~grouped["ci_upper"].isin([np.inf, -np.inf])) & (~grouped["ci_lower"].isin([np.inf, -np.inf]))

ages = grouped.loc[valid_idx, "age"].values
mean_alpha = grouped.loc[valid_idx, "mean_alpha"].values
ci_upper = grouped.loc[valid_idx, "ci_upper"].values
ci_lower = grouped.loc[valid_idx, "ci_lower"].values

# Create smooth x values
ages_smooth = np.linspace(ages.min(), ages.max(), 500)

# Fit splines (degree k=5)
mean_spline = make_interp_spline(ages, mean_alpha, k=5)
ci_upper_spline = make_interp_spline(ages, ci_upper, k=5)
ci_lower_spline = make_interp_spline(ages, ci_lower, k=5)

# Evaluate splines
mean_smooth = mean_spline(ages_smooth)
ci_upper_smooth = ci_upper_spline(ages_smooth)
ci_lower_smooth = ci_lower_spline(ages_smooth)

# ------------ Plot ------------
plt.figure(figsize=(14, 8))
plt.plot(ages_smooth, mean_smooth, label="Smoothed Mean", color="blue")
plt.fill_between(ages_smooth, ci_lower_smooth, ci_upper_smooth, color="skyblue", alpha=0.4, label="95% CI")
plt.scatter(ages, mean_alpha, color="blue", s=10, label="Mean Alpha Power")

plt.xlabel("Age")
plt.ylabel("Mean Alpha Power (µV²/Hz)")
plt.title("Mean Absolute Alpha Power vs. Age (EO Epochs) with 95% Confidence Interval")
plt.legend()
plt.grid(True)
plt.xticks(np.arange(0,100,10))
plt.yticks(np.arange(-2,9,1))
plt.tight_layout()
plt.show()

# Plotting relative mean alpha for 60 eyes open epochs per subject

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mne.time_frequency import psd_array_welch
from scipy.interpolate import make_interp_spline

# ------------ Load metadata ------------
metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str)

# ------------ Compute mean relative alpha power for random epochs ------------
relative_alpha_by_subject = {}
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"  # Update this path

for subject_id in top_60_EO_epochs_per_subject:
    try:
        set_path = f"{set_folder}/{subject_id}_epoched.set"
        epochs = mne.io.read_epochs_eeglab(set_path, verbose='ERROR')
        selected_data = epochs.get_data()[top_60_EO_epochs_per_subject[subject_id]]

        # Alpha power (8–13 Hz)
        psds_alpha, _ = psd_array_welch(
            selected_data,
            sfreq=epochs.info["sfreq"],
            fmin=8, fmax=13,
            n_fft=200,
            verbose=False
        )
        alpha_power = psds_alpha.sum(axis=-1).mean()  # sum over freq bins, mean over epochs & channels

        # Total power (1–40 Hz)
        psds_total, _ = psd_array_welch(
            selected_data,
            sfreq=epochs.info["sfreq"],
            fmin=1, fmax=40,
            n_fft=200,
            verbose=False
        )
        total_power = psds_total.sum(axis=-1).mean()

        # Relative alpha power
        relative_alpha = alpha_power / total_power
        relative_alpha_by_subject[subject_id] = relative_alpha

    except Exception as e:
        print(f" Could not process {subject_id}: {e}")


In [ ]:
# ------------ Merge relative alpha values with metadata ------------
alpha_df = pd.DataFrame({
    "subject_id": list(relative_alpha_by_subject.keys()),
    "relative_alpha_power": list(relative_alpha_by_subject.values())
})

metadata_subset = metadata[metadata["subject_id"].isin(alpha_df["subject_id"])]

alpha_df["subject_id"] = alpha_df["subject_id"].astype(str)
metadata_subset["subject_id"] = metadata_subset["subject_id"].astype(str)

merged = pd.merge(alpha_df, metadata_subset, on="subject_id")

# Filter out extreme relative alpha values (optional, adjust threshold as needed)
merged = merged[merged["relative_alpha_power"] <= 1.0]

# ------------ Group by age ------------
grouped = merged.groupby("age").agg(
    mean_rel_alpha=("relative_alpha_power", "mean"),
    std=("relative_alpha_power", "std"),
    N=("relative_alpha_power", "count")
).reset_index()

grouped["sem"] = grouped["std"] / np.sqrt(grouped["N"])
grouped["ci_upper"] = grouped["mean_rel_alpha"] + 1.96 * grouped["sem"]
grouped["ci_lower"] = grouped["mean_rel_alpha"] - 1.96 * grouped["sem"]

# Remove NaNs/Infs for smoothing
valid_idx = (~grouped["ci_upper"].isna()) & (~grouped["ci_lower"].isna()) & \
            (~grouped["ci_upper"].isin([np.inf, -np.inf])) & (~grouped["ci_lower"].isin([np.inf, -np.inf]))

ages = grouped.loc[valid_idx, "age"].values
mean_rel_alpha = grouped.loc[valid_idx, "mean_rel_alpha"].values
ci_upper = grouped.loc[valid_idx, "ci_upper"].values
ci_lower = grouped.loc[valid_idx, "ci_lower"].values

# Create smooth x values
ages_smooth = np.linspace(ages.min(), ages.max(), 500)

# Fit splines (degree k=5)
mean_spline = make_interp_spline(ages, mean_rel_alpha, k=5)
ci_upper_spline = make_interp_spline(ages, ci_upper, k=5)
ci_lower_spline = make_interp_spline(ages, ci_lower, k=5)

# Evaluate splines
mean_smooth = mean_spline(ages_smooth)
ci_upper_smooth = ci_upper_spline(ages_smooth)
ci_lower_smooth = ci_lower_spline(ages_smooth)

# ------------ Plot ------------
plt.figure(figsize=(14, 5))
plt.plot(ages_smooth, mean_smooth, label="Smoothed Mean Relative Alpha", color="blue")
plt.fill_between(ages_smooth, ci_lower_smooth, ci_upper_smooth, color="skyblue", alpha=0.4, label="95% CI")
plt.scatter(ages, mean_rel_alpha, color="blue", s=10, label="Mean Relative Alpha")

plt.xlabel("Age")
plt.ylabel("Mean Relative Alpha Power")
plt.title("Mean Relative Alpha Power vs. Age (EO Epochs) with 95% Confidence Interval")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


# Plotting absolute mean alpha power in the occipital channels for eyes open epochs 

In [ ]:
import mne
from mne.time_frequency import psd_array_welch
import numpy as np

alpha_by_subject_occipital = {}

for subject_id in list(top_60_EO_epochs_per_subject.keys()):
    print(f"Processing alpha for {subject_id}...")

    set_path = f"G:/ChristianMusaeus/Preprocessed_setfiles/{subject_id}_epoched.set"

    try:
        epochs = mne.io.read_epochs_eeglab(set_path, verbose='ERROR')
        selected_data = epochs.get_data()[top_60_EO_epochs_per_subject[subject_id]]  # (60, channels, timepoints)

        psds, freqs = psd_array_welch(
            selected_data,
            sfreq=epochs.info["sfreq"],
            fmin=8, fmax=13,
            n_fft=128,
            verbose=False
        )
        mean_power_per_channel = psds.mean(axis=-1)
        mean_power_over_epochs = mean_power_per_channel.mean(axis = 0)
        alpha_value = mean_power_over_epochs[8:10].mean() * 1e12

        alpha_by_subject_occipital[subject_id] = alpha_value

    except Exception as e:
        print(f" Could not process {subject_id}: {e}")


In [ ]:
# ------------ Load metadata and merge ------------

metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str)

alpha_df = pd.DataFrame({
    "subject_id": list(alpha_by_subject_occipital.keys()),
    "alpha_power_occipital": list(alpha_by_subject_occipital.values())
})

metadata_subset = metadata[metadata["subject_id"].isin(alpha_df["subject_id"])]

alpha_df["subject_id"] = alpha_df["subject_id"].astype(str)
metadata_subset["subject_id"] = metadata_subset["subject_id"].astype(str)

merged = pd.merge(alpha_df, metadata_subset, on="subject_id")

# Filter out extreme values if needed
merged = merged[merged["alpha_power_occipital"] <= 20]

# ------------ Group by age ------------

grouped = merged.groupby("age").agg(
    MeanAlpha=("alpha_power_occipital", "mean"),
    Std=("alpha_power_occipital", "std"),
    Count=("alpha_power_occipital", "count")
).reset_index()

grouped["SEM"] = grouped["Std"] / np.sqrt(grouped["Count"])
grouped["Upper"] = grouped["MeanAlpha"] + 1.96 * grouped["SEM"]
grouped["Lower"] = grouped["MeanAlpha"] - 1.96 * grouped["SEM"]

# Remove NaNs and inf values for smoothing
grouped_clean = grouped.replace([np.inf, -np.inf], np.nan).dropna(subset=["age", "MeanAlpha", "Lower", "Upper"])

# ------------ Prepare for smoothing ------------

x = grouped_clean["age"].values
y = grouped_clean["MeanAlpha"].values
y_lower = grouped_clean["Lower"].values
y_upper = grouped_clean["Upper"].values

x_smooth = np.linspace(x.min(), x.max(), 1000)

spline_mean = make_interp_spline(x, y, k=5)
spline_lower = make_interp_spline(x, y_lower, k=5)
spline_upper = make_interp_spline(x, y_upper, k=5)

y_smooth = spline_mean(x_smooth)
y_lower_smooth = spline_lower(x_smooth)
y_upper_smooth = spline_upper(x_smooth)

# ------------ Plot ------------

plt.figure(figsize=(12, 5))
plt.scatter(x, y, color='blue', s=30, label="Mean Alpha Power (Occipital)")
plt.plot(x_smooth, y_smooth, color='blue', label="Smoothed Mean")
plt.fill_between(x_smooth, y_lower_smooth, y_upper_smooth, color='skyblue', alpha=0.4, label="95% CI")

plt.xlabel("Age")
plt.ylabel("Mean Absolute Alpha Power (µV²/Hz)")
plt.title("Mean Absolute Alpha Power vs. Age (Occipital Channels) with 95% Confidence Interval")
plt.grid(True)
plt.xticks(np.arange(0, 100, 10))
plt.legend()
plt.tight_layout()
plt.show()

# Regression model eyes closed 

In [ ]:
import mne
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# ------------ LOAD DATA ------------
with open("top_epochs_per_subject.pkl", "rb") as f:
    top_epochs_per_subject = pickle.load(f)

top_epochs_per_subject = {str(k).strip(): v for k, v in top_epochs_per_subject.items()}

metadata = pd.read_csv("metadata_time_filtered.csv")
metadata["subject_id"] = metadata["subject_id"].astype(str).str.strip()

metadata = metadata[metadata["age"] < 90]

all_subjects = [s for s in top_epochs_per_subject.keys() if s in metadata["subject_id"].values]

random.seed(42)
test_subjects = random.sample(all_subjects, 500)
train_subjects = [s for s in all_subjects if s not in test_subjects]

# ------------ FEATURE EXTRACTION ------------
def extract_combined_features(subject_id, epoch_indices, set_folder):
    path = f"{set_folder}/{subject_id}_epoched.set"
    epochs = mne.io.read_epochs_eeglab(path, verbose='ERROR')
    data = epochs.get_data()[epoch_indices]
    sfreq = epochs.info["sfreq"]

    bands = {
        "delta": (1, 4),
        "theta": (4, 8),
        "alpha": (8, 13),
        "beta": (13, 30),
    }
    
    band_powers = []
    relative_powers = []

    total_psds, total_freqs = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=1, fmax=45, n_fft=128, verbose=False
    )
    total_power = total_psds.sum(axis=-1)

    mean_psd = total_psds.mean(axis=(0,1))
    freqs_log = np.log10(total_freqs)
    psd_log = np.log10(mean_psd)

    model = LinearRegression()
    model.fit(freqs_log.reshape(-1,1), psd_log)
    spectral_slope = model.coef_[0]

    psds_alpha, freqs_alpha = mne.time_frequency.psd_array_welch(
        data, sfreq=sfreq, fmin=8, fmax=13, n_fft=128, verbose=False
    )
    mean_alpha_psd = psds_alpha.mean(axis=(0,1))
    alpha_peak_idx = mean_alpha_psd.argmax()
    alpha_peak_freq = freqs_alpha[alpha_peak_idx]

    for band_name, (fmin, fmax) in bands.items():
        psds_band, _ = mne.time_frequency.psd_array_welch(
            data, sfreq=sfreq, fmin=fmin, fmax=fmax, n_fft=128, verbose=False
        )
        abs_power = psds_band.mean(axis=(0,2))
        band_powers.extend(abs_power)
        
        rel_power = (psds_band.sum(axis=-1) / total_power).mean()
        relative_powers.append(rel_power)
    
    features = np.concatenate([band_powers, relative_powers, [spectral_slope, alpha_peak_freq]])
    return features

# ------------ PREPARE TRAINING DATA ------------
set_folder = "G:/ChristianMusaeus/Preprocessed_setfiles"

X_train, y_train = [], []
X_test, y_test = [], []

for subj_id in train_subjects:
    try:
        epoch_inds = top_epochs_per_subject[subj_id]
        features = extract_combined_features(subj_id, epoch_inds, set_folder)
        age = metadata.loc[metadata["subject_id"] == subj_id, "age"].values[0]
        X_train.append(features)
        y_train.append(age)
    except Exception as e:
        print(f" Error processing train subject {subj_id}: {e}")

for subj_id in test_subjects:
    try:
        epoch_inds = top_epochs_per_subject[subj_id]
        features = extract_combined_features(subj_id, epoch_inds, set_folder)
        age = metadata.loc[metadata["subject_id"] == subj_id, "age"].values[0]
        X_test.append(features)
        y_test.append(age)
    except Exception as e:
        print(f" Error processing test subject {subj_id}: {e}")

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train_scaled, y_train)

preds = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"\n Test Mean Absolute Error: {mae:.3f}")
print(f" Test R^2 Score: {r2:.3f}")
